In [1]:
  #!/usr/bin/env python
#Numcosmo import
try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm


# General imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.table import Table
import pandas as pd

# DC2 catalog-related imports
import FoFCatalogMatching
import GCRCatalogs
from GCR import GCRQuery

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [2]:
#A list with all DC2 catalogs available

GCRCatalogs.get_available_catalog_names(include_default_only=False)

['base5000_v1.1.1',
 'base5000_v1.1.1_9431_9812',
 'base5000_v1.1.1_9556',
 'base5000_v1.1.1_image',
 'base5000_v1.1.1_small',
 'base5000_v1.1.1_z_0_1',
 'base5000_v1.1.1_z_1_2',
 'baseDC2_snapshot_z0.15_v0.1',
 'baseDC2_snapshot_z0.15_v0.1_small',
 'baseDC2_snapshot_z1.01_v0.1',
 'baseDC2_v1.1_9431_9812',
 'baseDC2_v1.1_9556',
 'baseDC2_v1.1_image',
 'buzzard',
 'buzzard_high-res',
 'buzzard_high-res_v1.1',
 'buzzard_test',
 'buzzard_v2.0.0_3',
 'buzzard_v2.0.0_4',
 'buzzard_v2.0.0_test',
 'cosmoDC2',
 'cosmoDC2_v1.0',
 'cosmoDC2_v1.0_10194_10452',
 'cosmoDC2_v1.0_8786_9049',
 'cosmoDC2_v1.0_9050_9430',
 'cosmoDC2_v1.0_9431_9812',
 'cosmoDC2_v1.0_9556',
 'cosmoDC2_v1.0_9813_10193',
 'cosmoDC2_v1.0_image',
 'cosmoDC2_v1.0_image_addon_knots',
 'cosmoDC2_v1.0_small',
 'cosmoDC2_v1.1.4',
 'cosmoDC2_v1.1.4_image',
 'cosmoDC2_v1.1.4_image_addon_knots',
 'cosmoDC2_v1.1.4_image_combined_agn',
 'cosmoDC2_v1.1.4_image_with_photoz_calib',
 'cosmoDC2_v1.1.4_image_with_photozs_flexzboost_v1',
 'co

In [3]:
#We are loading the cosmoDC2 catalalog with photoz
redmapper_cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

#Taking some important information about the fiducial cosmological model and survey region
cosmo_dc2 = redmapper_cat.cosmology
sky_area = redmapper_cat.sky_area

#Listing all quantities in the choosen catalog
np.sort(redmapper_cat.list_all_quantities())

array(['cluster_id', 'cluster_id_member', 'dec', 'dec_cen_0', 'dec_cen_1',
       'dec_cen_2', 'dec_cen_3', 'dec_cen_4', 'dec_member', 'id_cen_0',
       'id_cen_1', 'id_cen_2', 'id_cen_3', 'id_cen_4', 'id_member',
       'mag_g_lsst_member', 'mag_i_lsst_member', 'mag_r_lsst_member',
       'mag_y_lsst_member', 'mag_z_lsst_member', 'magerr_g_lsst_member',
       'magerr_i_lsst_member', 'magerr_r_lsst_member',
       'magerr_y_lsst_member', 'magerr_z_lsst_member', 'maskfrac',
       'p_cen_0', 'p_cen_1', 'p_cen_2', 'p_cen_3', 'p_cen_4', 'p_member',
       'pfree_member', 'ra', 'ra_cen_0', 'ra_cen_1', 'ra_cen_2',
       'ra_cen_3', 'ra_cen_4', 'ra_member', 'redshift', 'redshift_err',
       'redshift_true_cg', 'redshift_true_member', 'refmag_err_member',
       'refmag_member', 'richness', 'richness_err', 'scaleval',
       'theta_i_member', 'theta_r_member'], dtype='<U20')

In [15]:
z_min = 0
z_max = 1
richness_min = 0
richness_max = 5.5
data = redmapper_cat.get_quantities(['redshift','richness','richness_err','ra','dec'])



richness = data['richness']
richness_err = data['richness_err']
redshift = data['redshift']
ra = data['ra']
dec = data['dec']

ncountdata = {'richness': np.log(richness), 'richness_err': np.log(richness_err),'redshift': redshift,'ra':ra,'dec':dec}
ncountdata = pd.DataFrame(ncountdata)

richness_mask_min = (richness_min <= ncountdata['richness'])
richness_mask_max = (richness_max >= ncountdata['richness'] )

z_mask_min = (z_min <=ncountdata['redshift'])
z_mask_max = (z_max >=ncountdata['redshift'])

mask =  z_mask_min & z_mask_max & richness_mask_min & richness_mask_max

ncountdata = ncountdata[mask]




N_cl = len(ncountdata)
print(f'There are {N_cl} halos with Richness= [{np.exp(richness_min):.2e},{np.exp(richness_max):.2e}] Msun and z=[{z_min},{z_max}] in {redmapper_cat.sky_area} deg2')
display(ncountdata)



There are 38705 halos with Richness= [1.00e+00,2.45e+02] Msun and z=[0,1] in 439.78987 deg2


,richness,richness_err,redshift,ra,dec
0,5.250736,1.320947,0.513513,52.520436,-25.717996
1,5.188101,1.306557,0.571270,56.920377,-27.204341
3,5.120004,1.302292,0.635185,58.563624,-27.429962
4,4.717737,1.213890,0.442303,58.355198,-26.867687
5,4.648611,1.084195,0.466972,57.765757,-28.119102
...,...,...,...,...,...
45070,1.717804,0.408361,0.594197,68.391998,-45.515891
45074,1.781440,0.233527,0.471931,69.518360,-44.856106
45075,1.664387,0.210837,0.501281,70.472970,-44.819294
45078,1.611006,-0.107732,0.211521,71.592885,-44.833270
